### Setup Scene graph and image of map

In [17]:
import os
import json
import pprint
import re
from PIL import Image as PILImage
from Prompts import *
import pickle as pkl
import yaml
from utils.config import *
from utils.models import *
import copy
import subprocess
import xml.etree.ElementTree as ET
import ipywidgets a s widgets
from IPython.display import display
model = GPTModel(config = dict(
    MODEL_NAME = MODEL
))

with open(os.path.join('locations',LOCATION,'scene_graph.json'),'r') as f:
    scene_graph = json.load(f)

scgraph = utils.SceneGraph(scene_graph)
encoded_img = utils.encode_image(os.path.join('locations',LOCATION,'scene_graph.png'))
node_types = []
edge_types = []
for node in scgraph.get_parent_nodes():
    if scgraph.graph.nodes[node]['type'] not in node_types:
        node_types.append(scgraph.graph.nodes[node]['type'])

for edge in scgraph.graph.edges:
    if scgraph.graph.edges[edge]['type'] not in edge_types:
        edge_types.append(scgraph.graph.edges[edge]['type'])

### 1. Querying LLM for Scenario

In [18]:
scQ = ScenarioQuery()
scQ_full_prompt = scQ.get_full_prompt(
    context=CONTEXT,
    task=TASK,
    rough_scenario=ROUGH_SCENARIO,
    location = LOCATION_DESC
)

In [19]:
utils.pretty_print_conversation(scQ_full_prompt)

system: [{'type': 'text', 'text': 'Provide outputs strictly in JSON format'}]

user: [{'type': 'text', 'text': '\n                                      \nSocial Navigation is a complex skill for a robot to accomplish and the appropriateness of the behavior of a robot is highly dependent on the task and the social context. \nThus a robot’s social navigation capabilities must be thoroughly tested, and this is done by evaluating the robot’s behavior in a number of scenarios in a variety of contexts.\nYou are a scenario designer. Your task is to generate scenarios to test the social navigation capabilities of a robot.\nA Social Navigation [Scenario] is defined by:\n    1. Scenario Description: very detailed description of the scenario. WHAT happens in the scenario and WHERE the scenario takes place. WHERE the robot and humans are located.\n    2. Human Behavior:  how human interacts with the robot when it is visible, for e.g. Human 1 is scared of the robot and asks it to stop, Human 2 does

In [20]:
if QUERY_SC: 
    proposals = []
    for i in range(NUM_SCENARIO_PROPOSALS):
        print("Querying LLM for scenario")
        scq_response_json = model.get_response(messages = scQ_full_prompt,format = "json_object",expected_keys=scQ.required_output_keys)
        proposals.append(scq_response_json)
if USE_HANDCRAFTED_SCENARIO:
        print('USING HANDCRAFTED SCENARIO')
        print('Scenario Description:')
        scenario_desc = scenario_desc_hc
        num_humans = num_humans_hc
        behav_desc = behav_desc_hc

if LOAD_SC:
    print("Loading prior scenario response")
    with open(os.path.join(SAVE_DIR,'response_sc.json'),'r') as f:
        scq_response_json = json.load(f)
    
    scq_response_json = {k.lower().replace('_','').replace(' ',''): v for k,v in scq_response_json.items()}
    scenario_desc = scq_response_json['scenariodescription']
    behav_desc =  {k.lower().replace('_','').replace(' ',''): v for k,v in scq_response_json['humanbehavior'].items()}
    num_humans =  scq_response_json['numberofhumans']

Querying LLM for scenario
Querying LLM for scenario
Querying LLM for scenario
Querying LLM for scenario
Querying LLM for scenario


In [21]:
rb = widgets.RadioButtons(
        options=list(range(1,NUM_SCENARIO_PROPOSALS+1)),
        description='Choose a scenario from the following:',
        layout={'width': 'max-content'}, 
        disabled=False
    )
for i,proposal in enumerate(proposals):
    print(f"""{i+1}: {proposal['scenariodescription']}""")

1: The robot is navigating through the warehouse to guide two humans to a safe exit. The scenario starts with the robot in the central Passageway, moving towards the north exit where the humans are initially located near different racks. Human 1 is in the northeast corner of the warehouse, searching for a safe exit. Human 2 is closer to the west side, looking around anxiously.
2: The robot is navigating through the passageways of the small warehouse, where it encounters two humans. Human 1 is located near an intersection where several racks meet, while Human 2 is in an open packaging area. The robot's task is to guide both humans toward a safety exit located at the end of one of the passageways.
3: The robot is navigating through the warehouse, starting from the intersection near the packaging area and moving towards the safety exit located at the far end of the warehouse. It encounters two humans: Human 1 is near the packaging area, frantically looking around, while Human 2 is at a di

In [12]:
display(rb)

RadioButtons(description='Choose a scenario from the following:', layout=Layout(width='max-content'), options=…

In [16]:
if QUERY_SC:
    scq_response_json = proposals[rb.value-1]
    if SAVE_SC_RESPONSE:
        print("Saving scenario response")
        with open(os.path.join(SAVE_DIR,f'response_sc.json'),'w') as f:
            json.dump(scq_response_json,f)
    print('USING GENERATED SCENARIO')
    scenario_desc = scq_response_json['scenariodescription']
    num_humans = scq_response_json['numberofhumans']
    behav_desc = scq_response_json['humanbehavior']
    
    print(scenario_desc)
    print(f'Number of Humans:{num_humans}')
    print(f'Behaviors:{behav_desc}')

Saving scenario response
USING GENERATED SCENARIO
The robot is positioned at the center of the warehouse near an intersection. The robot's task is to guide humans to a designated safety zone located at the north-eastern corner of the warehouse. Human 1 and Human 2 are initially present in separate open packaging areas on the western side. The robot proceeds to move towards each human, guide them to the safety zone, passing through the racks and intersections.
Number of Humans:2
Behaviors:{'Human 1': 'Human 1 is initially packing items and does not notice the robot at first. When the robot becomes visible, Human 1 follows the robot to the safety zone while glancing around occasionally.', 'Human 2': 'Human 2 is attempting to find an exit and moving around the packaging area in distress. When the robot becomes visible, Human 2 stops looking around and starts following the robot to the safety zone quickly.'}


### 2. Querying LLM for Human and Robot Trajectories

In [ ]:
flq = FLocationQuery()

flq_full_prompt = flq.get_full_prompt(
    scene_graph = str(scene_graph),
    node_types = ','.join(node_types),
    edge_types = ','.join(edge_types),
    encoded_img = encoded_img,
    sc_desc = scenario_desc
)

In [ ]:
utils.pretty_print_conversation(flq_full_prompt)

In [ ]:
if QUERY_TRAJ:
    print("Querying LLM for Trajectories")
    valid_trajectories = 0
    all_trajectories_valid = False
    payload = flq_full_prompt.copy()
    while not all_trajectories_valid:
        retries = -1
        payload = flq_full_prompt.copy()
        while retries <=3: #retry until trajectories are valid (max 3)
            all_trajectories_valid = True
            flq_response_json = model.get_response(messages = payload,format = 'json_object',expected_keys=flq.required_output_keys)
            if not isinstance(flq_response_json,dict):
                retries+=1
                continue
            trajectories = flq_response_json['trajectories']
            groupids = flq_response_json['groupids']
            traj_valid = True
            #test connectivity
            for k,v in trajectories.items():
                print(f'Proposed Trajectory:{v}')
                traj_valid,errors = scgraph.isvalidtrajectory(v)
                if not traj_valid: #requery LLM with error message
                    print("Disconnected Trajectory Output, Retrying")
                    valid_trajectories=0
                    all_trajectories_valid = False
                    reply = f"""
                            You made the following mistakes:
                            <ERRORS>
                            Retry and Return the answer in the same JSON format.
                            """
                    error_string = ""
                    for err in errors:
                        error_string+=f"There is no edge connecting {err[0]} and {err[1]}!"
                        
                    reply = reply.replace('<ERRORS>',error_string)
                    payload.append(
                        {
                            "role":"assistant",
                            "content":[
                                {
                                    "type":"text",
                                    "text": str(flq_response_json)
                                }
                            ]
                        }    
                        )
                    payload.append({
                            "role": "user", 
                            "content": [
                                {
                                    "type":"text",
                                    "text": reply
                                }
                                    ]
                                }
                                )
                    
                    retries+=1
                    break
                
                  

                if traj_valid and traj_intersecting:
                    valid_trajectories+=1
        
            if all_trajectories_valid:
                break
    groupids =  {k.lower().replace('_','').replace(' ',''): v for k,v in flq_response_json['groupids'].items()}
    trajectories =  {k.lower().replace('_','').replace(' ',''): v for k,v in flq_response_json['trajectories'].items()}

    if SAVE_TRAJ_RESPONSE:
        print("Saving trajectory response")
        #with open('responses/reponse_traj.json','w') as f:
        with open(os.path.join(SAVE_DIR,'response_traj.json'),'w') as f:
            json.dump(flq_response_json,f)
                
else:
    assert LOAD_TRAJ_RESPONSE == True
    print("Loading prior fine location response")
    with open(os.path.join(SAVE_DIR,'response_traj.json'),'r') as f:
        flq_response_json = json.load(f)
    
    flq_response_json = {k.lower().replace('_','').replace(' ',''): v for k,v in flq_response_json.items()}
    groupids =  {k.lower().replace('_','').replace(' ',''): v for k,v in flq_response_json['groupids'].items()}
    trajectories =  {k.lower().replace('_','').replace(' ',''): v for k,v in flq_response_json['trajectories'].items()}


world_trajectories = {}        
for k,v in trajectories.items():
    world_trajectories[k] = []
    for l in v:
        world_trajectories[k].append(utils.pix2world(scgraph.graph.nodes[l]['pos']))

In [ ]:
print(f'OUTPUT TRAJECTORIES:{trajectories}')
print(f'GROUP:{flq_response_json["groupids"]}')
print(f'REASONING:{flq_response_json["reasoning"]}')

world_trajectories = {'robot': [[4.785913853317808, -6.094902723735679],
  [1.7561117578579681, -6.282840466926341],
  [1.6895227008148943, 0.9715564202333073],
  [-1.340279394644945, -2.3925291828795885]],
 'human1': [[1.7561117578579681, -6.282840466926341]]}

In [ ]:
#writing to HuNavSim files
print("ADDING TRAJECTORIES TO SIM YAML FILES")
agents_yaml = {'hunav_loader': {'ros__parameters': {'map': LOCATION,
   'publish_people': True,
   'agents': []}}}
blank_human = {'id': None,
    'skin': 0,
    'behavior': 0,
    'group_id': -1,
    'max_vel': 1.5,
    'radius': 0.4,
    'init_pose': {'x': None, 'y': None, 'z': 1.25, 'h': 0.0},
    'goal_radius': 0.3,
    'cyclic_goals': False,
    'goals': [],
    }
agents = {}


for i in range(len(trajectories.keys())-1):
    agents_yaml['hunav_loader']['ros__parameters']['agents'].append(f'agent{i}')
    agents[f'agent{i}'] = copy.deepcopy(blank_human)
    agents[f'agent{i}']['id'] = i
    agents[f'agent{i}']['behavior'] = 7+i
    agents[f'agent{i}']['group_id'] = groupids[f'human{i+1}']
    for j,g in enumerate(world_trajectories[f'human{i+1}']):
        if j == 0:
            agents[f'agent{i}']['init_pose'] = {
                'x':g[0],
                'y':g[1],
                'z':1.25,
                'h':0.0,
            }
            if len(world_trajectories[f'human{i+1}']) <= 1:
                agents[f'agent{i}']['goals'].append(f'g{1}')
                agents[f'agent{i}'][f'g{1}'] = {
                    'x':g[0],
                    'y':g[1],
                    'h':1.25
                }
        else:
            agents[f'agent{i}']['goals'].append(f'g{j}')
            agents[f'agent{i}'][f'g{j}'] = {
                'x':g[0],
                'y':g[1],
                'h':1.25
            }
agents_yaml['hunav_loader']['ros__parameters'].update(agents)

In [ ]:
with open(HUNAV_SIM_AGENTS_FILE,'w') as f:
    yaml.dump(agents_yaml,f)

with open(os.path.join(HUNAV_GAZEBO_WRAPPER_DIR,'config','robot.yaml'),'w') as f:
    yaml.dump({
    'x_pose': world_trajectories['robot'][0][0],
    'y_pose': world_trajectories['robot'][0][1],
    'roll': 0.0,
    'pitch': 0.0,
    'yaw': 3.14
},f)

### 3. Querying LLM for Human Behaviors

In [ ]:
behav_desc =  {k.lower().replace('_','').replace(' ',''): v for k,v in scq_response_json['humanbehavior'].items()}
print(behav_desc)

In [ ]:
for i in range(len(list(behav_desc.keys()))):
    print(i)
    btq = BTQuery()
    if QUERY_BT:
        print(f'Querying for Human {i+1}')
        #print(behav_desc[f'human{i+1}']['Behavior Towards Robot'])
        
        btq_full_prompt = btq.get_full_prompt(behavior = behav_desc[f'human{i+1}'])#['Behavior Towards Robot'])
        btq_response_json = model.get_response(messages = btq_full_prompt,format = 'json_object',expected_keys=btq.required_output_keys,node_library = node_library)
        if SAVE_BT_RESPONSE:
            print(f"Saving BT {i+1} response")
            #with open(f'responses/reponse_bt_{i+1}.json','w') as f:
            with open(os.path.join(SAVE_DIR,f'response_bt_{i+1}.json'),'w') as f:
                json.dump(btq_response_json,f)
    else:
        assert LOAD_BT_RESPONSE == True
        print(f'Loading prior BT {i+1} response')
        with open(os.path.join(SAVE_DIR,f'response_bt_{i+1}.json'),'w') as f:
            btq_response_json = json.load(f)
    
    print('================Proposed Behavior:================') 
    print(json.dumps(btq_response_json,indent=4))
    print("Continuing...")
    bt_xml = btq_response_json['tree']
    for k,v in btq_response_json.items():
        if 'custom' in str.lower(k):
            custom_node_requests.append(v)
    print(os.path.join(HUNAV_SIM_BT_FOLDER,f'LLMBT_{i}.xml'))
    with open(os.path.join(HUNAV_SIM_BT_FOLDER,f'LLMBT_{i}.xml'),'w') as f:
        f.write(bt_xml)
    print(f"Wrote BT to LLMBT_{i}.xml")


In [ ]:
xml = print(bt_xml)

In [ ]:
behav_desc.keys()